# 概要

`动量效应是指过去收益较高的股票，在未来一段时间内仍具有相对较好的收益`。 动量效应普遍存在于国外的股票市场中，但在A股市场中，动量效应却难寻踪迹。 众所周知，无论是`短端`涨跌幅因子还是`长端`涨跌幅因子，在A股市场均呈现为显著的**反转效应**。A 股市场具有强反转这一特征属性，使得有效的动量因子变得稀少而珍贵。

考虑到长端涨跌幅的反转效应弱于短端涨跌幅的反转效应，传统研究也尝试从长端涨跌幅之中切割出具有动量的部分。`传统切割方案基于时间维度，使用长端涨跌幅与短端涨跌幅作差，以希望切割后的部分能够呈现出动量效应。`但整体上动量效应并不显著。

通过上文的分析，构造有效动量因子的关键在于：探寻合适的维度将动量与反转进行切割区分。在讨论涨跌幅因子合适的切割维度之前，我们先从交易行为角度对反转效应进行分析。行为金融学理论表明，反转效应是投资者的交易行为引发的市场现象，股价反转很大程度上与投资者过度反应有关。资金的过度涌入与过度流出， 一定程度上都会导致股价的反转。因此我们思考：不同交易活跃程度下，动量和反转效应是否存在差异性？

我们从交易行为维度出发，尝试构造衡量交易活跃程度的指标来对涨跌幅因子进行切割。我们测试了**日度振幅**等指标的切割效果，最终给出了基于日度振幅的涨跌幅因子切割方案。具体切割步骤如下:

|步骤|说明|
|--|--|
|步骤1|对选定股票，回溯取其近N个交易日的数据;|
|步骤2|计算股票每日的**振幅**(HIGH/LOW-1);|
|步骤3|选择振幅较低的N/2交易日,涨跌幅加总,记为A因子;|
|步骤4|选择振幅较高的N/2交易日,涨跌幅贾总,记为B因子;|

我们选择不同窗口下的涨跌幅因子作为切割对象，对切割得到的A因子和B因子进行了测试。从而说明基于日度振幅的涨跌幅因子切割方案，能够从长端涨跌幅中切割出有效的动量因子。

## 一、数据导入与整理

In [74]:
#导入库

import numpy as np
import pandas as pd
import scipy.stats as st

import matplotlib as mpl
import matplotlib.pyplot as plt


mpl.rcParams['font.family']='serif' # pd.plot中文
# 用来正常显示负号
mpl.rcParams['axes.unicode_minus'] = False
# 图表主题
plt.style.use('seaborn')

In [7]:
#导入数据
data_origin=pd.read_excel(r".\data\4.xlsx",engine="openpyxl")
#删去最后两行空值和wind引用
data_origin.drop(data_origin.tail(2).index,inplace=True) 

In [14]:
#计算振幅
data_origin["振幅(%)"]=(data_origin["最高价(元)"]/data_origin["最低价(元)"]-1)*100
data_origin.head()

,代码,简称,日期,前收盘价(元),开盘价(元),最高价(元),最低价(元),收盘价(元),成交量(股),成交金额(元),涨跌(元),涨跌幅(%),均价(元),振幅(%)
0,000001.SZ,平安银行,2018-03-19,1237.43,1239.5606,1258.6962,1234.2452,1257.6331,80853872,947664744,20.1987,1.6323,11.7207,1.981049
1,000001.SZ,平安银行,2018-03-20,1257.63,1248.0653,1262.9486,1245.9392,1256.5701,77615002,916858475,-1.0631,-0.0845,11.8129,1.365187
2,000001.SZ,平安银行,2018-03-21,1256.57,1270.3902,1288.4627,1259.7593,1265.0748,144510951,1736437401,8.5047,0.6768,12.016,2.278483
3,000001.SZ,平安银行,2018-03-22,1265.07,1265.0748,1272.5164,1235.3083,1239.5606,98427842,1155741055,-25.5141,-2.0168,11.742,3.012050
4,000001.SZ,平安银行,2018-03-23,1239.56,1195.9740,1206.6049,1160.8921,1205.5418,182569075,2037734400,-34.0188,-2.7444,11.1614,3.937730


In [26]:
#输出基本信息
print("股票池总数：",len(data_origin["简称"].unique()))
print("回测时间  ：",data_origin["日期"].iloc[0].strftime('%Y-%m-%d'),"至",data_origin["日期"].iloc[-1].strftime('%Y-%m-%d'),"共",len(data_origin["日期"].unique()),"天")

股票池总数： 300
回测时间  ： 2018-03-19 至 2023-03-17 共 1215 天


## 二、计算因子值

In [91]:
#计算因子值，包括基本动量因子、A因子和B因子

def cal_factor(N):
    stock_names=data_origin["简称"].unique()

    data_factors=pd.DataFrame()

    count=0
    for stock_name in stock_names:
        data_stock=data_origin[data_origin["简称"]==stock_name].copy()
        data_stock["最近"+str(N)+"日涨跌幅(%)"]=(data_stock["收盘价(元)"]/data_stock["收盘价(元)"].shift(N)-1)*100
        data_stock

        i = 0
        df=pd.DataFrame(columns=["涨跌幅(%)","振幅(%)"])
        df_ab=pd.DataFrame(columns=["A因子","B因子"])
        for ind, row in data_stock.iterrows():
            if i < N-1:
                df.loc[ind]=[row["涨跌幅(%)"],row["振幅(%)"]]
                value_a=None
                value_b=None
            elif i >= N-1:
                if i > N-1:
                    df.drop(index = [ind-N],inplace = True)
                df.loc[ind]=[row["涨跌幅(%)"],row["振幅(%)"]]
                df=df.sort_values(by="振幅(%)")
                value_a=df["涨跌幅(%)"].head(int(N/2)).sum()
                value_b=df["涨跌幅(%)"].tail(int(N/2)).sum()
            df_ab.loc[ind]=[value_a,value_b]
            i += 1
        df_fab=pd.concat([data_stock,df_ab],axis=1)
        data_factors=pd.concat([data_factors,df_fab])
        count+=1
        print(count,end="")
    return data_factors

data_factors=cal_factor(160)


In [92]:
data_factors120=cal_factor(120)

123456789101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899100101102103104105106107108109110111112113114115116117118119120121122123124125126127128129130131132133134135136137138139140141142143144145146147148149150151152153154155156157158159160161162163164165166167168169170171172173174175176177178179180181182183184185186187188189190191192193194195196197198199200201202203204205206207208209210211212213214215216217218219220221222223224225226227228229230231232233234235236237238239240241242243244245246247248249250251252253254255256257258259260261262263264265266267268269270271272273274275276277278279280281282283284285286287288289290291292293294295296297298299300

In [98]:
data_factors60=cal_factor(60)

123456789101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899100101102103104105106107108109110111112113114115116117118119120121122123124125126127128129130131132133134135136137138139140141142143144145146147148149150151152153154155156157158159160161162163164165166167168169170171172173174175176177178179180181182183184185186187188189190191192193194195196197198199200201202203204205206207208209210211212213214215216217218219220221222223224225226227228229230231232233234235236237238239240241242243244245246247248249250251252253254255256257258259260261262263264265266267268269270271272273274275276277278279280281282283284285286287288289290291292293294295296297298299300

In [100]:
data_factors30=cal_factor(30)

123456789101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899100101102103104105106107108109110111112113114115116117118119120121122123124125126127128129130131132133134135136137138139140141142143144145146147148149150151152153154155156157158159160161162163164165166167168169170171172173174175176177178179180181182183184185186187188189190191192193194195196197198199200201202203204205206207208209210211212213214215216217218219220221222223224225226227228229230231232233234235236237238239240241242243244245246247248249250251252253254255256257258259260261262263264265266267268269270271272273274275276277278279280281282283284285286287288289290291292293294295296297298299300

## 三、计算IC值

In [82]:
#计算下一日收益率
def next_ret(df:pd.DataFrame)->pd.DataFrame:
    df["下一日涨跌幅(%)"]=(df["收盘价(元)"].shift(-1)/df["收盘价(元)"]-1)*100
    return df

# 计算ic
def cal_ic(df:pd.DataFrame)->pd.DataFrame:
    f = ["最近160日涨跌幅(%)","A因子","B因子"]
    ic = df[f].apply(lambda x:st.spearmanr(x,df['下一日涨跌幅(%)'])[0])
    return ic

# IC计算
data_factors_1=data_factors.dropna(axis=0, how='any').groupby("简称").apply(next_ret)
ic_df = data_factors_1.dropna(axis=0, how='any').groupby('日期').apply(cal_ic)
ic_df.mean()

最近160日涨跌幅(%)    0.008513
A因子             0.020146
B因子            -0.004740
dtype: float64

In [96]:
# 计算ic
def cal_ic(df:pd.DataFrame)->pd.DataFrame:
    f = ["最近120日涨跌幅(%)","A因子","B因子"]
    ic = df[f].apply(lambda x:st.spearmanr(x,df['下一日涨跌幅(%)'])[0])
    return ic
# IC计算
data_factors_1=data_factors120.dropna(axis=0, how='any').groupby("简称").apply(next_ret)
ic_df = data_factors_1.dropna(axis=0, how='any').groupby('日期').apply(cal_ic)
ic_df.mean()

最近120日涨跌幅(%)    0.003411
A因子             0.017489
B因子            -0.007500
dtype: float64

In [99]:
# 计算ic
def cal_ic(df:pd.DataFrame)->pd.DataFrame:
    f = ["最近60日涨跌幅(%)","A因子","B因子"]
    ic = df[f].apply(lambda x:st.spearmanr(x,df['下一日涨跌幅(%)'])[0])
    return ic
# IC计算
data_factors_1=data_factors60.dropna(axis=0, how='any').groupby("简称").apply(next_ret)
ic_df = data_factors_1.dropna(axis=0, how='any').groupby('日期').apply(cal_ic)
ic_df.mean()

最近60日涨跌幅(%)   -0.006317
A因子            0.012343
B因子           -0.012736
dtype: float64

In [102]:
# 计算ic
def cal_ic(df:pd.DataFrame)->pd.DataFrame:
    f = ["最近30日涨跌幅(%)","A因子","B因子"]
    ic = df[f].apply(lambda x:st.spearmanr(x,df['下一日涨跌幅(%)'])[0])
    return ic
# IC计算
data_factors_1=data_factors30.dropna(axis=0, how='any').groupby("简称").apply(next_ret)
ic_df = data_factors_1.dropna(axis=0, how='any').groupby('日期').apply(cal_ic)
ic_df.mean()

最近30日涨跌幅(%)   -0.009884
A因子            0.004111
B因子           -0.013899
dtype: float64

## 四、结果分析

**IC值均值：**

|回看天数|动量因子|A因子(振幅小)|B因子(振幅大)|
|--|--|--|--|
|160日|0.008513|0.020146|-0.004740|
|120日|0.003411|0.017489|-0.007500|
|60日|-0.006317|0.012343|-0.012736|
|30日|-0.009884|0.004111|-0.013899|

可以看出，在不同的回看天数下，动量因子的IC值有正有负，振幅小的A动量因子具有正的IC值，振幅大的B动量因子具有负的IC值。在较长的回看天数下，振幅小的A动量因子IC值绝对值较大；在较短的回看天数下，振幅大的B动量因子IC值绝对值较大。不论在哪种情况下，振幅的大小都很好地切割了传统的动量因子，使得基础的动量因子在简单的调整下有着更好的解释能力。

本研究的缺陷在于统计得到的各因子IC值均值都不超过0.03，总体来说解释能力较差。另外，研究只选取了几个特殊的回看天数取值，没有对更大范围和更精细的回看天数进行统计，也没有通过改变振幅大小的切割比例(目前设定为默认的50%对50%)来测试不同的因子，参数的敏感性分析有很大的进步空间。这些问题主要是受限于电脑性能和对于python数据结构的不熟悉，目前跑一次因子计算的程序需要半小时左右，无法承担较大规模的参数分析。在数据结构方面，在每次划分振幅小和振幅大的因子时使用优先级队列应该可以大大加快计算速度，避免每次迭代时的重新排序，不过在多次尝试反复报错下放弃了，望老师体谅！